Time and memory measures were collected right from the beginning of the code.

In [ ]:
import time
import tracemalloc
start_time = time.time()
tracemalloc.start()
import os
import pandas as pd

Importing a csv file of communities and crime after removing the features from raw dataset.

In [ ]:
os.chdir('C:/Users/bitel/PycharmProjects/CIND820_Project/CIND820_Project')
crime = pd.read_csv('cleanedcommunitiescrime.csv', sep=',')

First column counts the number of observations. Removed first column.<br>
Removed target variable column from predictor variables

In [ ]:
x = crime.drop(crime.columns[[0, 95]], axis = 1)

The target variable "ViolentCrimesPerPop" was divided into 5 quantile categorical variables.

In [ ]:
y = crime['ViolentCrimesPerPop']
y = pd.qcut(y, q=5, labels=['Very Low Crime', 'Low Crime','Medium Crime', 'High Crime', 'Very High Crime'])

Data was split into training (0.7) and test (0.3) data. The following preprocessing procedures and modelling will occur on the training data.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y, test_size= 0.3, random_state=225)

One feature, 'OtherperCap' was an object data type but was supposed to be a numeric value.<br>
Change feature to numeric value.

In [ ]:
x_train['OtherPerCap'] = pd.to_numeric(x_train['OtherPerCap'], errors='coerce')

Converted the missing/null values into column median values.

In [ ]:
x_train = x_train.fillna(x_train.median())

The feature selection filter method Kendall tau correlation coefficient was applied to the preprocessed training dataset.<br>
The top 10 features were selected.

In [ ]:
from scipy.stats import kendalltau
import operator
kendall_list = []
for col in x_train.columns:
    tau, p = kendalltau(x_train[col], y_train)
    kendall_list.append((col, tau))
selection = sorted(kendall_list, key=lambda x_train: abs(x_train[1]), reverse=True)
kendall_list = selection[:10]
selected_features = list(map(operator.itemgetter(0), kendall_list))
newx_train = x_train[selected_features]
print("Top 10 Selected Features:", selected_features)

Use the trained data to determine it's accuracy scores based on zero cross validation.<br>
The model used to classify the trained data was the Support Vector Classifier (SVC).<br>
Accuracy measure will be used to determine variance score.

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
model = SVC()
model.fit(newx_train, y_train)
print("zero-fold scores:", classification_report(y_train, model.predict(newx_train)))

k-fold cross validation was used to evaluate the Support Vector Classifier (SVC) model against 20 split samples.<br>
The average accuracy score was determined as a performance metric.<br>
The average variance from the 10-folds was calculated using the average accuracy score.

In [ ]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
model = SVC()
score = cross_val_score(model, newx_train, y_train, scoring= 'accuracy', cv=cv, n_jobs=-1)
mean_score = sum(score)/30
print("R10-fold mean accuracy score:", mean_score)
print("R10-Fold accuracy variance:", mean_score*(1-mean_score))

Computed matthew_corrcoef to determine MCC score of the test data.

In [ ]:
from sklearn.metrics import matthews_corrcoef
model.fit(newx_train, y_train)
newx_test = x_test[selected_features]
y_pred = model.predict(newx_test)
mcc = matthews_corrcoef(y_test, y_pred)
print("MCC score:", mcc)

In addition, anoher classification report was used to evaluate the performance of the model against the test data.

In [ ]:
print("Test data performance report:", classification_report(y_test, model.predict(newx_test)))

End of time and memory tracking.

In [ ]:
snapshot = tracemalloc.take_snapshot()
end_time = time.time()
tracemalloc.stop()
memory = snapshot.statistics('lineno')
for stat in memory[:10]:
    print("Memory Used:", stat)
print("Duration of execution (sec):", end_time - start_time)